In [11]:
#!pip install --upgrade httpx httpx-sse PyJWT
from langchain_community.chat_models import ChatZhipuAI
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_core.callbacks.manager import CallbackManager
from langchain_core.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
import os

os.environ["ZHIPUAI_API_KEY"] = "d9ae4d81e22cb9483f5b4d875ba2d1c1.0QNkBJCwDX7rLdI9"

In [12]:
chat = ChatZhipuAI(
    model= "glm-4-plus",
    temperature=0.5,
    streaming=True,
    callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]),
)

In [13]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

# Define a new graph
workflow = StateGraph(state_schema=MessagesState)


# Define the function that calls the model
def call_model(state: MessagesState):
    response = chat.invoke(state["messages"])
    return {"messages": response}


# Define the (single) node in the graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [14]:
config = {"configurable": {"thread_id": "abc123"}}

In [15]:
query = "Hi! I'm Bob."

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()  # output contains all messages in state

Hi Bob! How can I assist you today?================================== Ai Message ==================================

Hi Bob! How can I assist you today?


In [16]:
query = "What's my name?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

Your name is Bob! How can I help you today, Bob?================================== Ai Message ==================================

Your name is Bob! How can I help you today, Bob?


In [18]:
# from IPython.display import Image, display
#
# # try:
# display(Image(workflow.get_graph().draw_mermaid_png()))
# # except Exception:
# #     # This requires some extra dependencies and is optional
# #     print('error')
# #     pass

AttributeError: 'StateGraph' object has no attribute 'get_graph'